## MCP Museum Art Display Gallery

In [9]:
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
import os
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "artic": {
        "transport": "stdio",
        "command": "npx",
        "args": ["-y", "artic-mcp"],
        "env": {
            # helps silence npm/npx chatter if any
            "NPM_CONFIG_LOGLEVEL": "silent",
            "npm_config_loglevel": "silent",
        },
    }
})

tools = await client.get_tools()
print([t.name for t in tools])


['search-by-title', 'get-artwork-by-id', 'full-text-search', 'search-for-artist', 'get-artwork-by-artist']


In [18]:
import json

tool_map = {t.name: t for t in tools}
fts = tool_map["full-text-search"]

schema = fts.args_schema  # already a dict
print(json.dumps(schema, indent=2))


{
  "type": "object",
  "properties": {
    "query": {
      "type": "string",
      "description": "The term to search the metadata for."
    },
    "limit": {
      "type": "number",
      "default": 10,
      "description": "The number of resources to return per page."
    },
    "page": {
      "type": "number",
      "default": 1,
      "description": "The page of results to return. Used for pagination."
    }
  },
  "required": [
    "query"
  ],
  "additionalProperties": false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [36]:
res = await fts.ainvoke({"query": "moon", "limit": 10})
res


[{'type': 'text',
  'text': "Title: Moonrise\nArtwork ID: 64754\nThumbnail alt text: A work made of oil on canvas.\nScore: 104.62242\n\n-----\nTitle: Square from a 'Crazy Quilt'\nArtwork ID: 11791\nThumbnail alt text: A work made of silk, in a variety of weaves including cut velvet weave; pieced; embroidered in silk threads in a variety of stitches; backed with cotton, plain weave; lined with silk, satin weave.\nScore: 93.52393\n\n-----\nTitle: Square from a 'Crazy Quilt'\nArtwork ID: 11790\nThumbnail alt text: A work made of silk, tabby, satin, velvet & twill weaves. some have warp-face, extra pattern wefts, printing, or moiré pattern, cisele velvet\n\nembroidered in silk floss & 2 ply twist. (backed with tabby cotton after piecing.) embroidery thru 3 layers\n\nstitches: herrinbone, feather, buttonhole, cross, satin, fishbone knot, & variations with line stitch.\nScore: 90.04958\n\n-----\nTitle: Moon Jar\nArtwork ID: 158472\nThumbnail alt text: A work made of porcelain with glaze.\nSc

In [37]:
import re

text = res[0]["text"]
ids = re.findall(r"Artwork ID:\s*(\d+)", text)
ids = [int(x) for x in ids]
ids


[64754, 11791, 11790, 158472, 23323, 20634, 87479, 140735, 89985, 20823]

In [54]:
tool_map = {t.name: t for t in tools}
get_by_id = tool_map["get-artwork-by-id"]

one = await get_by_id.ainvoke({"id": ids[0]})
print(one[0]['text'])


Title: Moonrise
Artist: George Inness (American, 1825–1894)
Artist ID: 35061
Description: No description available
Image ID: 4425984b-e241-6413-1404-cdac0fb06518
Place of Origin: United States
Dimensions: 76.5 × 64.1 cm (30 1/8 × 25 1/4 in.)
Medium: Oil on canvas
Credit Line: Edward B. Butler Collection
Department: Arts of the Americas
Is On View: No
Main Reference Number: 1911.39
Has not been viewed much: No
Date Start: 1891
Date End: 1891
Date: 1891
Fiscal Year: N/A
Is Public Domain: Yes
Gallery: N/A
Artwork Type: Painting
Artist Title: George Inness
Artist Titles: George Inness
Style Title: N/A
Image URL: https://www.artic.edu/iiif/2/4425984b-e241-6413-1404-cdac0fb06518/full/843,/0/default.jpg



In [46]:
import re

blocks = one  # this is your tool output for a single artwork
text = blocks[0]["text"]

m = re.search(r"Image URL:\s*(\S+)", text)
image_url = m.group(1) if m else None
image_url


'https://www.artic.edu/iiif/2/103681af-b044-e195-3439-3583a07539fd/full/843,/0/default.jpg'

In [41]:
from IPython.display import Image, display

display(Image(url=image_url, width=450))


In [55]:
import re
from IPython.display import Image, display
from langchain_core.tools import ToolException
import requests

tool_map = {t.name: t for t in tools}
get_by_id = tool_map["get-artwork-by-id"]

def extract_image_url(blocks):
    txt = blocks[0]["text"] if blocks and blocks[0].get("type") == "text" else ""
    m = re.search(r"Image URL:\s*(\S+)", txt)
    return m.group(1) if m else None

def extract_title(blocks):
    txt = blocks[0]["text"] if blocks and blocks[0].get("type") == "text" else ""
    m = re.search(r"Title:\s*(.+)", txt)
    return m.group(1).strip() if m else "Untitled"

def show_with_fallback(url):
    for u in (url, url.replace("/full/843,/", "/full/600,/")):
        try:
            r = requests.get(u, timeout=15)
            r.raise_for_status()
            display(Image(url=u, width=350))
            return
        except Exception:
            pass
    print("Skipping image (403/other error).")

shown = 0
for art_id in ids:
    try:
        blocks = await get_by_id.ainvoke({"id": art_id})
        url = extract_image_url(blocks)
        title = extract_title(blocks)
        print(f"{art_id}: {title}")
        if url:
            #display(Image(url=url, width=350))
            show_with_fallback(url)
            shown += 1
        else:
            print(f"{art_id}: no image url")
    except ToolException as e:
        #print(f"{art_id}: skipped (server schema error: {e})")
        print(f"{art_id}: skipped (server schema validation error)")
    continue


    if shown >= 10:
        break


64754: Moonrise


11791: skipped (server schema validation error)
11790: skipped (server schema validation error)
158472: skipped (server schema validation error)
23323: Morning-Glories, Rabbits, and Moon


20634: Blue Moon (A) [Aoi tsuki (A)]


87479: The Assumption of the Virgin


140735: Amulet of Thoth


89985: skipped (server schema validation error)
20823: Chrysanthemums and the Rising Moon


In [64]:
# --- Artist search -> list artworks (titles + IDs) ---
#artist_name = "Vincent van Gogh"   # change this each time: "Picasso", "Monet", etc.
#artist_name = "Pablo Picasso"
artist_name = "Monet"
max_artworks = 10

tool_map = {t.name: t for t in tools}
search_artist = tool_map["search-for-artist"]
get_by_artist = tool_map["get-artwork-by-artist"]

# 1) Find artist candidates
artist_res = await search_artist.ainvoke({"name": artist_name})

# The tool returns a list of content blocks; first block is text
artist_text = artist_res[0]["text"]
print(artist_text)

# 2) Extract the first Artist ID from the text
import re
m = re.search(r"Artist ID:\s*(\d+)", artist_text)
if not m:
    raise ValueError("No Artist ID found in search-for-artist output.")
artist_id = int(m.group(1))
print(f"\nUsing Artist ID: {artist_id}")

# 3) Get artworks by that artist
arts_res = await get_by_artist.ainvoke({"id": artist_id, "limit": max_artworks})
arts_text = arts_res[0]["text"]
print("\n" + arts_text)

# 4) Parse artwork IDs for your existing image-display cell
artwork_ids = [int(x) for x in re.findall(r"Artwork ID:\s*(\d+)", arts_text)]
print("\nArtwork IDs:", artwork_ids)


Title: Claude Monet
Artist ID: 35809
Score: 12.179682

-----
Title: Musée Marmottan Monet
Artist ID: 93557
Score: 10.925768

Pagination Info
Total: 2
Total Pages: 1
Current Page: 1


Using Artist ID: 35809

Title: Stacks of Wheat (End of Summer)
Artwork ID: 64818
Thumbnail alt text: Painting composed of short, dense brushstrokes depicts two domed stacks of wheat that cast long shadows on a field. The angled light indicates either a rising or setting sun.
Score: 233.09041

-----
Title: Water Lilies
Artwork ID: 16568
Thumbnail alt text: Painting of a pond seen up close spotted with thickly painted pink and white water lilies and a shadow across the top third of the picture.
Score: 61.6069

-----
Title: Arrival of the Normandy Train, Gare Saint-Lazare
Artwork ID: 16571
Thumbnail alt text: Loosely painted image of an open-air train station. On the right, a parked train gives off an enormous plumb of white smoke, making the scene look as though it were full of clouds. A huddled mass of bare

In [65]:
shown = 0
for art_id in artwork_ids:
    try:
        blocks = await get_by_id.ainvoke({"id": art_id})
        url = extract_image_url(blocks)
        title = extract_title(blocks)
        print(f"{art_id}: {title}")
        if url:
            #display(Image(url=url, width=350))
            show_with_fallback(url)
            shown += 1
        else:
            print(f"{art_id}: no image url")
    except ToolException as e:
        #print(f"{art_id}: skipped (server schema error: {e})")
        print(f"{art_id}: skipped (server schema validation error)")
    continue


    if shown >= 10:
        break


64818: Stacks of Wheat (End of Summer)


16568: Water Lilies


16571: Arrival of the Normandy Train, Gare Saint-Lazare


14598: The Beach at Sainte-Adresse


14620: Cliff Walk at Pourville


87088: Water Lily Pond


81537: Bordighera


14624: Stacks of Wheat (End of Day, Autumn)


16564: Branch of the Seine near Giverny (Mist)


81539: On the Bank of the Seine, Bennecourt
